In [1]:
import os
import re
import piexif

In [2]:
srcDirectory = '/media/jean-phi/EOS_DIGITAL'
tgtDirectory = './images'

In [3]:
def _checkTrailingSlash(path='./'):
    '''
    Check if a slash is presen tat the end of o path.
    If not add it.
    
    parameters:
    path as a string
    
    return value:
    path with  a trailing slash
    '''
    if path[-1] != '/':
        return path + '/'
    else:
        return path

def _isTargetedFile(filename='', targetFiles=['CR2']):
    '''
    Check if a file name has the correct extension
    
    parameters:
    filename as a string (with complete path or not
    targetFiles as an array of all valid extention.
    
    return value:
    True if the filename is correct.
    False otherwise.
    '''
    #targetFiles=['CR2', 'JPG']
    targetFilePattern = targetFiles[0]
    for t in targetFiles[1:]:
        targetFilePattern += '|' + t
    #print(os.path.basename(filename))
    regexPattern = '^.*\.({})$'.format(targetFilePattern)
    #print(regexPattern)
    filePattern = re.compile(regexPattern, re.IGNORECASE)
    if filePattern.match(filename):
        return True
        #print('OK')
    else:
        return False
        #print('KO')
    
def listFiles(srcPath='.'):
    '''
    list all image files with their path
    
    parameters:
    source path to begin
    
    return value
    a list of all files with their path
    '''
    filesList = []
    filesInDir = os.listdir(srcPath)
    for f in filesInDir:
        fileWithPath = _checkTrailingSlash(srcPath) + f
        if os.path.isfile(fileWithPath):
            #print('{} is  a file'.format(fileWithPath))
            if _isTargetedFile(fileWithPath):
                filesList.append(fileWithPath)
        elif os.path.isdir(fileWithPath):
            #print('{} is a directory'.format(fileWithPath))
            filesList += listFiles(fileWithPath)
        else:
            print('file {} not found'.format(fileWithPath))
    return filesList

In [4]:
def getImageParameters(imagePath):
    '''
    Get parameters of the image (in Exif)
    
    Parameters:
    path of the image as string
    
    return value:
    A dictionary with image parameters
       number of the image file
       camera model
       ISO
       Aperture
       Exposition
       Focal length
       Date and time
    '''
    imageParameters = {}
    #Load exif information from the file
    exifData = piexif.load(imagePath)
    #Get the image number
    imageNumberRe = re.compile('[0-9]+')
    imageNumber = imageNumberRe.search(os.path.basename(imagePath)).group()
    imageParameters['Image Number'] = imageNumber
    
    imageParameters['Camera'] = exifData['0th'][0x110]
    imageParameters['ISO'] = exifData['Exif'][0x8827]
    aperture = exifData['Exif'][0x829d]
    imageParameters['Aperture'] = aperture[0] / aperture[1]
    exposureTime = exifData['Exif'][0x829a]
    imageParameters['Exposure Time'] = exposureTime[0] / exposureTime[1]
    focalLength = exifData['Exif'][0x920a]
    imageParameters['Focal Length'] = focalLength[0] / focalLength[1]
    imageParameters['Date Time Original'] = exifData['Exif'][0x9003]
    
    return imageParameters

In [5]:
listFiles(srcDirectory)

['/media/jean-phi/EOS_DIGITAL/DCIM/100CANON/IMG_0679.CR2',
 '/media/jean-phi/EOS_DIGITAL/DCIM/100CANON/IMG_0680.CR2',
 '/media/jean-phi/EOS_DIGITAL/DCIM/100CANON/IMG_0681.CR2',
 '/media/jean-phi/EOS_DIGITAL/DCIM/100CANON/IMG_0682.CR2',
 '/media/jean-phi/EOS_DIGITAL/DCIM/100CANON/IMG_0683.CR2',
 '/media/jean-phi/EOS_DIGITAL/DCIM/100CANON/IMG_0684.CR2',
 '/media/jean-phi/EOS_DIGITAL/DCIM/100CANON/IMG_0685.CR2',
 '/media/jean-phi/EOS_DIGITAL/DCIM/100CANON/IMG_0686.CR2',
 '/media/jean-phi/EOS_DIGITAL/DCIM/100CANON/IMG_0687.CR2',
 '/media/jean-phi/EOS_DIGITAL/DCIM/100CANON/IMG_0688.CR2',
 '/media/jean-phi/EOS_DIGITAL/DCIM/100CANON/IMG_0689.CR2',
 '/media/jean-phi/EOS_DIGITAL/DCIM/100CANON/IMG_0690.CR2',
 '/media/jean-phi/EOS_DIGITAL/DCIM/100CANON/IMG_0691.CR2',
 '/media/jean-phi/EOS_DIGITAL/DCIM/100CANON/IMG_0692.CR2',
 '/media/jean-phi/EOS_DIGITAL/DCIM/100CANON/IMG_0693.CR2',
 '/media/jean-phi/EOS_DIGITAL/DCIM/100CANON/IMG_0694.CR2',
 '/media/jean-phi/EOS_DIGITAL/DCIM/100CANON/IMG_0695.CR2

In [6]:
_checkTrailingSlash('test/')

'test/'

In [7]:
_isTargetedFile('/media/jean-phi/EOS_DIGITAL/DCIM/100CANON/IMG_0681.CR2')

True

In [8]:
_isTargetedFile('/media/jean-phi/EOS_DIGITAL/DCIM/100CANON/IMG_0681.jpg', targetFiles=['cr2', 'jpg'])

True

In [9]:
getImageParameters('/media/jean-phi/EOS_DIGITAL/DCIM/100CANON/IMG_0681.CR2')

{'Aperture': 5.6,
 'Camera': b'Canon EOS 1200D',
 'Date Time Original': b'2018:07:29 15:16:37',
 'Exposure Time': 0.003125,
 'Focal Length': 35.0,
 'ISO': 200,
 'Image Number': '0681'}